# BIGDATA FOR DYNAMICID

## Clone DynamicID Repository
- Gốc: https://github.com/traananhdat/DynamicID
- Mới: https://github.com/LongTruongPhamHai/bigdata-for-dynamicid

In [1]:
# Xóa thư mục cũ nếu đã tồn tại
!rm -rf /kaggle/working/bigdata-for-dynamicid

# Clone repo github
!git clone https://github.com/LongTruongPhamHai/bigdata-for-dynamicid

Cloning into 'bigdata-for-dynamicid'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 453 (delta 44), reused 50 (delta 22), pack-reused 381 (from 1)
Receiving objects: 100% (453/453), 50.73 MiB | 49.38 MiB/s, done.
Resolving deltas: 100% (152/152), done.


## Train SAA

In [ ]:
# Tải thư viện
!pip uninstall -y diffusers huggingface_hub transformers accelerate peft

!pip install \
  diffusers==0.27.2 \
  transformers==4.41.2 \
  accelerate==0.27.2 \
  peft==0.10.0 \
  huggingface_hub==0.23.4

In [ ]:
# Kiểm tra thư viện
!pip show diffusers transformers accelerate peft

In [ ]:
!accelerate launch \
  --num_processes 1 \
  /kaggle/working/bigdata-for-dynamicid/train_SAA.py \
  --pretrained_model_name_or_path runwayml/stable-diffusion-v1-5 \
  --image_encoder_path openai/clip-vit-large-patch14 \
  --data_root_path /kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset \
  --data_json_file /kaggle/working/bigdata-for-dynamicid/data.json \
  --mixed_precision fp16 \
  --train_batch_size 1 \
  --num_train_epochs 10

In [ ]:
# Nén file checkpoint
!rm /kaggle/working/checkpoint-2000.zip
# !zip -r /kaggle/working/checkpoint-2000.zip /kaggle/working/sd-ip_adapter/checkpoint-2000

In [ ]:
# Xóa 3 file checkpoint không cần thiết
!rm /kaggle/working/sd-ip_adapter/checkpoint-2000/optimizer.bin \
    /kaggle/working/sd-ip_adapter/checkpoint-2000/random_states_0.pkl \
    /kaggle/working/sd-ip_adapter/checkpoint-2000/scaler.pt

## Train IMR

### Tạo SAA.bin từ /sd-ip_adapter/checkpoint-2000/model.safetensors

In [ ]:
# Kiểm tra thư viện
!pip show torch

In [ ]:
import os
import torch
from safetensors.torch import load_file

ckpt_path = "/kaggle/input/saa-model-safetensors/saa-model.safetensors"
out_path = "/kaggle/working/models/SAA.bin"

ckpt = load_file(ckpt_path)

saa_state = {}

for k, v in ckpt.items():
    if "lora" not in k.lower():
        continue

    k = k.replace("unet.", "")
    k = k.replace(".processor.", ".")
    k = k.replace("transformer_blocks.0.", "")

    saa_state[k] = v.cpu()

assert len(saa_state) > 0, "❌ No SAA weights extracted"

os.makedirs(os.path.dirname(out_path), exist_ok=True)

torch.save(
    {
        "state_dict": saa_state,
        "rank": 4,          
        "type": "SAA"
    },
    out_path
)

print(f"✅ Exported {len(saa_state)} SAA weights → {out_path}")

### Chạy file create_IMR_cache.py

In [ ]:
# Tải thư viện
!pip install -U insightface onnxruntime-gpu

In [ ]:
import torch

saa = torch.load("/kaggle/working/models/SAA.bin", map_location="cpu")

print(saa.keys())
print("Rank:", saa["rank"])
print("Num params:", len(saa["state_dict"]))

for k, v in list(saa["state_dict"].items())[:3]:
    print(k, v.shape)

In [ ]:
!python /kaggle/working/bigdata-for-dynamicid/create_IMR_cache.py

In [ ]:
# Xóa file IMR_cache.zip nếu đã tồn tại
!rm /kaggle/working/IMR_cache.zip

# Nén toàn bộ thư mục cache thành file IMR_cache.zip
!zip -r /kaggle/working/IMR_cache.zip /kaggle/working/cache

### Tạo promt và landmark cho ảnh 
Model:
- BLIP: Tạo promt
- FaceMesh: Tạo landmark

In [ ]:
!pip install dlib opencv-python-headless pillow

In [ ]:
!rm /kaggle/working/shape_predictor_68_face_landmarks.dat.bz2
!rm /kaggle/working/shape_predictor_68_face_landmarks.dat

# tải file nén từ dlib.net
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

# giải nén file .bz2 thành .dat
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

!rm /kaggle/working/shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
!rm -rf /kaggle/working/dataset

In [ ]:
import os
import cv2
import random
import numpy as np
from PIL import Image
import dlib

# ================== PATH CONFIG ==================
BASE_INPUT_DIR = "/kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset"  # ảnh gốc chỉ đọc
OUTPUT_DIR = "/kaggle/working/dataset/base_image_dataset"  # nơi lưu landmark + prompt
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================== SETTINGS ==================
expressions = [
    "sad face", "happy expression", "neutral expression",
    "angry expression", "surprised look", "calm expression", "serious face"
]
poses = [
    "facing forward", "looking to the left", "looking to the right",
    "chin down", "chin up", "head slightly tilted", "three-quarter view"
]

TARGET_SIZE = (512, 512)  # resize về 512x512

def generate_face_prompt():
    return f"{random.choice(expressions)}, {random.choice(poses)}"

# ================== INIT LANDMARK MODEL ==================
predictor_path = "/kaggle/working/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# ================== PROCESS ==================
person_ids = sorted([d for d in os.listdir(BASE_INPUT_DIR)
                     if os.path.isdir(os.path.join(BASE_INPUT_DIR, d)) and d.isdigit()])

for pid in person_ids:
    person_input_dir = os.path.join(BASE_INPUT_DIR, pid)
    person_output_dir = os.path.join(OUTPUT_DIR, pid)
    os.makedirs(person_output_dir, exist_ok=True)

    img_files = [f for f in sorted(os.listdir(person_input_dir)) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

    # Giữ tối đa 35 ảnh / người, nhân bản nếu ít
    img_files = img_files[:35]
    while len(img_files) < 35:
        img_files.append(img_files[-1])

    for img_file in img_files:
        img_path = os.path.join(person_input_dir, img_file)
        img_id = os.path.splitext(img_file)[0]

        try:
            # --------- Load và resize ảnh gốc ---------
            img_pil = Image.open(img_path).convert("RGB")
            img_resized = img_pil.resize(TARGET_SIZE, Image.BICUBIC)
            img_output_path = os.path.join(person_output_dir, img_file)
            img_resized.save(img_output_path)

            # --------- Landmark với dlib ---------
            img_np = np.array(img_resized)
            gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

            faces = detector(gray)
            if len(faces) > 0:
                face = faces[0]  # chỉ lấy khuôn mặt đầu tiên
                shape = predictor(gray, face)
                landmarks = [(pt.x, pt.y) for pt in shape.parts()]

                # Vẽ landmark thành ảnh đen trắng
                landmark_img = np.zeros((TARGET_SIZE[1], TARGET_SIZE[0]), dtype=np.uint8)
                for (x, y) in landmarks:
                    if 0 <= x < TARGET_SIZE[0] and 0 <= y < TARGET_SIZE[1]:
                        landmark_img[y, x] = 255

                landmark_save_path = os.path.join(person_output_dir, f"{img_id}_landmark.png")
                cv2.imwrite(landmark_save_path, landmark_img)

            # --------- Prompt ---------
            prompt_text = generate_face_prompt()
            txt_save_path = os.path.join(person_output_dir, f"{img_id}.txt")
            with open(txt_save_path, "w", encoding="utf-8") as f:
                f.write(prompt_text)

        except Exception as e:
            print(f"[SKIP] {img_path}: {e}")
            continue

print("✅ Hoàn tất: landmarks (dlib 68 điểm), ảnh 512x512 và prompt được lưu trong", OUTPUT_DIR)

In [ ]:
import os
import cv2
import random
import numpy as np
from PIL import Image
import dlib

# ================== PATH CONFIG ==================
BASE_INPUT_DIR = "/kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset"  # ảnh gốc chỉ đọc
OUTPUT_DIR = "/kaggle/working/dataset/base_image_dataset"  # nơi lưu landmark + prompt
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================== SETTINGS ==================
expressions = [
    "sad face", "happy expression", "neutral expression",
    "angry expression", "surprised look", "calm expression", "serious face"
]
poses = [
    "facing forward", "looking to the left", "looking to the right",
    "chin down", "chin up", "head slightly tilted", "three-quarter view"
]

IMG_SIZE = (512, 512)       # ảnh feed VAE/UNet
LANDMARK_SIZE = (64, 64)    # landmark feed keypoint_encoder/IMR

def generate_face_prompt():
    return f"{random.choice(expressions)}, {random.choice(poses)}"

# ================== INIT LANDMARK MODEL ==================
predictor_path = "/kaggle/working/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# ================== PROCESS ==================
person_ids = sorted([d for d in os.listdir(BASE_INPUT_DIR)
                     if os.path.isdir(os.path.join(BASE_INPUT_DIR, d)) and d.isdigit()])

for pid in person_ids:
    person_input_dir = os.path.join(BASE_INPUT_DIR, pid)
    person_output_dir = os.path.join(OUTPUT_DIR, pid)
    os.makedirs(person_output_dir, exist_ok=True)

    img_files = [f for f in sorted(os.listdir(person_input_dir)) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

    # Giữ tối đa 35 ảnh / người, nhân bản nếu ít
    img_files = img_files[:35]
    while len(img_files) < 35:
        img_files.append(img_files[-1])

    for img_file in img_files:
        img_path = os.path.join(person_input_dir, img_file)
        img_id = os.path.splitext(img_file)[0]

        try:
            # --------- Load và resize ảnh gốc (512x512) ---------
            img_pil = Image.open(img_path).convert("RGB")
            img_resized = img_pil.resize(IMG_SIZE, Image.BICUBIC)
            img_output_path = os.path.join(person_output_dir, img_file)
            img_resized.save(img_output_path)

            # --------- Landmark với dlib ---------
            img_np = np.array(img_resized)
            gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
            faces = detector(gray)

            if len(faces) > 0:
                face = faces[0]  # chỉ lấy khuôn mặt đầu tiên
                shape = predictor(gray, face)
                landmarks = [(pt.x, pt.y) for pt in shape.parts()]

                # Vẽ landmark thành ảnh đen trắng 512x512
                landmark_img = np.zeros(IMG_SIZE[::-1], dtype=np.uint8)
                for (x, y) in landmarks:
                    if 0 <= x < IMG_SIZE[0] and 0 <= y < IMG_SIZE[1]:
                        landmark_img[y, x] = 255

                # Resize landmark về 64x64
                landmark_img_small = cv2.resize(landmark_img, LANDMARK_SIZE, interpolation=cv2.INTER_NEAREST)
                landmark_save_path = os.path.join(person_output_dir, f"{img_id}_landmark.png")
                cv2.imwrite(landmark_save_path, landmark_img_small)

            # --------- Prompt ---------
            prompt_text = generate_face_prompt()
            txt_save_path = os.path.join(person_output_dir, f"{img_id}.txt")
            with open(txt_save_path, "w", encoding="utf-8") as f:
                f.write(prompt_text)

        except Exception as e:
            print(f"[SKIP] {img_path}: {e}")
            continue

print("✅ Hoàn tất: ảnh 512x512, landmark 64x64 và prompt được lưu trong", OUTPUT_DIR)

In [ ]:
# Xóa file dataset.zip nếu đã tồn tại
!rm /kaggle/working/dataset.zip

# Nén toàn bộ thư mục dataset thành file dataset.zip
!zip -r /kaggle/working/dataset.zip /kaggle/working/dataset

## Chạy file train_IMR.py

In [ ]:
# Tạo thư mục lưu model
!mkdir -p /kaggle/working/sd-v1-5

In [ ]:
!pip install torch

In [ ]:
!pip show torch

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# Tải model SD-v1-5 từ Hugging Face
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    dtype=torch.float16
)

# Lưu ra thư mục chuẩn diffusers
pipe.save_pretrained("/kaggle/working/sd-v1-5")

In [ ]:
import torch 
checkpoint = torch.load("/kaggle/working/bigdata-for-dynamicid/models/SAA.bin", map_location="cpu")
print(checkpoint.keys())

In [2]:
!python /kaggle/working/bigdata-for-dynamicid/train_IMR.py \
    --pretrained_model_name_or_path /kaggle/working/sd-v1-5 \
    --IMR_depth 1 \
    --num_tokens 16 \
    --resolution 512 \
    --save_steps 500 \
    --output_dir /kaggle/working/bigdata-for-dynamicid/output \
    --logging_dir logs \
    --learning_rate 1e-4 \
    --weight_decay 0.01 \
    --num_train_epochs 20 \
    --train_batch_size 4 \
    --dataloader_num_workers 2 \
    --mixed_precision fp16 \
    --report_to tensorboard

2025-12-24 14:15:25.737587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766585725.758995     483 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766585725.765578     483 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766585725.783138     483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766585725.783166     483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766585725.783169     483 computation_placer.cc:177] computation placer alr

## Cell cũ - Chỉ xem, không cần chạy

## Data Preprocess
### Rename image (Chạy trực tiếp trên máy)

In [ ]:
from pathlib import Path


def rename_images(directory: Path):
    files = [f for f in directory.iterdir() if f.is_file()]
    files.sort()  # sắp xếp để có thứ tự ổn định

    changed = 0
    for idx, f in enumerate(files, start=1):
        if idx > 35:
            break

        new_name = f"{idx:04d}.jpg"
        new_path = f.with_name(new_name)

        if new_path == f:
            continue  # đã đúng định dạng
        if new_path.exists():
            print(f"SKIP: Tồn tại sẵn {new_path.name}, bỏ qua {f.name}")
            continue

        f.rename(new_path)
        changed += 1
        print(f"RENAMED: {f.name} -> {new_path.name}")

    print(f"Done. Đã đổi tên {changed} file.")


if __name__ == "__main__":
    dir_path = Path(
        r"D:\DEV\.Projects\TLU\PTDL\DynamicID_fork_prv\dataset\base_image_dataset\2"
    ).resolve()
    rename_images(dir_path)

### FaceCrop Image (Đã xử lý và lưu vào dataset-for-dynamicid)
Model: MediaPipe Face Detection

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 mediapipe==0.10.31 opencv-python

In [ ]:
# Xóa thư mục nếu đã tồn tại
!rm -rf /kaggle/working/mediapipe_models

# Tạo thư mục mới
!mkdir -p /kaggle/working/mediapipe_models

# Tải model short-range
!wget \
https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/latest/blaze_face_short_range.tflite \
-O /kaggle/working/mediapipe_models/blaze_face_short_range.tflite

In [ ]:
# FaceCrop cho 1 thư mục con của dataset
import cv2
import os
from mediapipe import Image, ImageFormat
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ===== PATH =====
base_input_dir = "/kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset"
base_output_dir = "/kaggle/working/dataset/base_image_dataset/6"
short_model = "/kaggle/working/mediapipe_models/blaze_face_short_range.tflite"

# ===== CHỌN THƯ MỤC MUỐN XỬ LÝ =====
target_subfolder = "6"   # đổi tên thư mục bạn muốn crop ở đây

def load_detector(model_path):
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.FaceDetectorOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.IMAGE
    )
    return vision.FaceDetector.create_from_options(options)

# ===== LOAD MODEL =====
detector = load_detector(short_model)

# ===== PROCESS CHỈ MỘT THƯ MỤC =====
input_dir = os.path.join(base_input_dir, target_subfolder)
output_dir = os.path.join(base_output_dir, target_subfolder)

if os.path.isdir(input_dir):
    os.makedirs(output_dir, exist_ok=True)
    print(f"🔄 Processing folder: {target_subfolder}")

    for file in os.listdir(input_dir):
        if not file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img = cv2.imread(os.path.join(input_dir, file))
        if img is None:
            continue

        h, w, _ = img.shape
        mp_image = Image(
            image_format=ImageFormat.SRGB,
            data=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        )

        result = detector.detect(mp_image)

        if not result.detections:
            cv2.imwrite(os.path.join(output_dir, file), img)
            print(f"⚠️ No face detected, saved original: {file}")
            continue

        bbox = result.detections[0].bounding_box
        x1, y1 = int(bbox.origin_x), int(bbox.origin_y)
        bw, bh = int(bbox.width), int(bbox.height)
        x2, y2 = x1 + bw, y1 + bh

        x_pad = int(0.2 * bw)
        y_pad = int(0.3 * bh)

        x1 = max(0, x1 - x_pad)
        y1 = max(0, y1 - y_pad)
        x2 = min(w, x2 + x_pad)
        y2 = min(h, y2 + y_pad)

        crop = img[y1:y2, x1:x2]
        cv2.imwrite(os.path.join(output_dir, file), crop)

        print(f"✔️ Cropped: {file}")

print("✅ DONE ONE FOLDER")

In [ ]:
# FaceCrop toàn bộ thư mục trong dataset
import cv2
import os
from mediapipe import Image, ImageFormat
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ===== PATH =====
base_input_dir = "/kaggle/input/dataset-for-dynamicid/dataset/base_image_dataset"
base_output_dir = "/kaggle/working/dataset/base_image_dataset"
short_model = "/kaggle/working/mediapipe_models/blaze_face_short_range.tflite"

def load_detector(model_path):
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.FaceDetectorOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.IMAGE
    )
    return vision.FaceDetector.create_from_options(options)

# ===== LOAD MODEL =====
detector = load_detector(short_model)

# ===== PROCESS ALL SUBFOLDERS =====
for subfolder in sorted(os.listdir(base_input_dir)):
    input_dir = os.path.join(base_input_dir, subfolder)
    output_dir = os.path.join(base_output_dir, subfolder)

    if not os.path.isdir(input_dir):
        continue  # bỏ qua nếu không phải thư mục

    os.makedirs(output_dir, exist_ok=True)

    print(f"🔄 Processing folder: {subfolder}")

    for file in os.listdir(input_dir):
        if not file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img = cv2.imread(os.path.join(input_dir, file))
        if img is None:
            continue

        h, w, _ = img.shape
        mp_image = Image(
            image_format=ImageFormat.SRGB,
            data=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        )

        result = detector.detect(mp_image)

        if not result.detections:
            # ❌ Không nhận diện được mặt → lưu nguyên ảnh
            cv2.imwrite(os.path.join(output_dir, file), img)
            print(f"⚠️ No face detected, saved original: {file}")
            continue

        # lấy khuôn mặt đầu tiên
        bbox = result.detections[0].bounding_box
        x1, y1 = int(bbox.origin_x), int(bbox.origin_y)
        bw, bh = int(bbox.width), int(bbox.height)
        x2, y2 = x1 + bw, y1 + bh

        # mở rộng vùng crop vừa phải
        x_pad = int(0.2 * bw)
        y_pad = int(0.3 * bh)

        x1 = max(0, x1 - x_pad)
        y1 = max(0, y1 - y_pad)
        x2 = min(w, x2 + x_pad)
        y2 = min(h, y2 + y_pad)

        crop = img[y1:y2, x1:x2]
        cv2.imwrite(os.path.join(output_dir, file), crop)

        print(f"✔️ Cropped: {file}")

print("✅ DONE ALL FOLDERS")

In [ ]:
# Xóa file output.zip nếu đã tồn tại
!rm /kaggle/working/output.zip

# Nén toàn bộ thư mục output thành file output.zip
!zip -r /kaggle/working/output.zip /kaggle/working/dataset/base_image_dataset

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path

path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = path.read_text()

text = text.replace(
    "ip_tokens = self.image_proj_model(image_embeds)",
    """dummy_id = torch.zeros(
        image_embeds.shape[0],
        512,
        device=image_embeds.device,
        dtype=image_embeds.dtype
    )
    ip_tokens = self.image_proj_model(dummy_id, image_embeds)"""
)

path.write_text(text)
print("✅ Fixed ProjPlusModel forward call")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path
import re

path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = path.read_text()

pattern = re.compile(r"image_proj_model\s*=\s*ProjPlusModel\([\s\S]*?\)", re.MULTILINE)

replacement = """image_proj_model = ProjPlusModel(
        cross_attention_dim=unet.config.cross_attention_dim,
        id_embeddings_dim=512,
        clip_embeddings_dim=image_encoder.config.hidden_size,
        num_tokens=args.num_tokens,
    )"""

text = pattern.sub(replacement, text)
path.write_text(text)

print("✅ Fixed ProjPlusModel init")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path

file_path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = file_path.read_text()

old = "attn_procs[name] = SAProcessor()"

new = """attn_procs[name] = SAProcessor(
                hidden_size=hidden_size,
                rank=4,
                network_alpha=4,
            )"""

text = text.replace(old, new)
file_path.write_text(text)

print("✅ Fixed SAProcessor init")

In [ ]:
# Sửa code train_SAA.py (Đã sửa lại trong repo gihub => Không cần chạy cell này)
from pathlib import Path

file_path = Path("/kaggle/working/DynamicID/train_SAA.py")
text = file_path.read_text()

text = text.replace(
    "attn_procs[name].load_state_dict(weights)",
    "attn_procs[name].load_state_dict(weights, strict=False)"
)

file_path.write_text(text)
print("✅ Fixed load_state_dict(strict=False)")